## Fitting a diagonal covariance Gaussian mixture model to text data

In a previous assignment, we explored k-means clustering for a high-dimensional Wikipedia dataset. We can also model this data with a mixture of Gaussians, though with increasing dimension we run into two important issues associated with using a full covariance matrix for each component.
 * Computational cost becomes prohibitive in high dimensions: score calculations have complexity cubic in the number of dimensions M if the Gaussian has a full covariance matrix.
 * A model with many parameters require more data: observe that a full covariance matrix for an M-dimensional Gaussian will have M(M+1)/2 parameters to fit. With the number of parameters growing roughly as the square of the dimension, it may quickly become impossible to find a sufficient amount of data to make good inferences.

Both of these issues are avoided if we require the covariance matrix of each component to be diagonal, as then it has only M parameters to fit and the score computation decomposes into M univariate score calculations. Recall from the lecture that the M-step for the full covariance is:

\begin{align*}
\hat{\Sigma}_k &= \frac{1}{N_k^{soft}} \sum_{i=1}^N r_{ik} (x_i-\hat{\mu}_k)(x_i - \hat{\mu}_k)^T
\end{align*}

Note that this is a square matrix with M rows and M columns, and the above equation implies that the (v, w) element is computed by

\begin{align*}
\hat{\Sigma}_{k, v, w} &= \frac{1}{N_k^{soft}} \sum_{i=1}^N r_{ik} (x_{iv}-\hat{\mu}_{kv})(x_{iw} - \hat{\mu}_{kw})
\end{align*}

When we assume that this is a diagonal matrix, then non-diagonal elements are assumed to be zero and we only need to compute each of the M elements along the diagonal independently using the following equation. 

\begin{align*}
\hat{\sigma}^2_{k, v} &= \hat{\Sigma}_{k, v, v}  \\
&= \frac{1}{N_k^{soft}} \sum_{i=1}^N r_{ik} (x_{iv}-\hat{\mu}_{kv})^2
\end{align*}

In this section, we will use an EM implementation to fit a Gaussian mixture model with **diagonal** covariances to a subset of the Wikipedia dataset. The implementation uses the above equation to compute each variance term. 

We'll begin by importing the dataset and coming up with a useful representation for each article. After running our algorithm on the data, we will explore the output to see whether we can give a meaningful interpretation to the fitted parameters in our model.

**Note to Amazon EC2 users**: To conserve memory, make sure to stop all the other notebooks before running this notebook.

## Import necessary packages

The following code block will check if you have the correct version of GraphLab Create. Any version later than 1.8.5 will do. To upgrade, read [this page](https://turi.com/download/upgrade-graphlab-create.html).

In [1]:
import graphlab

'''Check GraphLab Create version'''
from distutils.version import StrictVersion
assert (StrictVersion(graphlab.version) >= StrictVersion('1.8.5')), 'GraphLab Create must be version 1.8.5 or later.'

We also have a Python file containing implementations for several functions that will be used during the course of this assignment.

In [2]:
from em_utilities import *

## Load Wikipedia data and extract TF-IDF features

Load Wikipedia data and transform each of the first 5000 document into a TF-IDF representation.

In [3]:
wiki = graphlab.SFrame('people_wiki.gl/').head(5000)
wiki['tf_idf'] = graphlab.text_analytics.tf_idf(wiki['text'])

This non-commercial license of GraphLab Create for academic use is assigned to damien.pretet@me.com and will expire on September 17, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1483177012.log


Using a utility we provide, we will create a sparse matrix representation of the documents. This is the same utility function you used during the previous assignment on k-means with text data.

In [4]:
tf_idf, map_index_to_word = sframe_to_scipy(wiki, 'tf_idf')

As in the previous assignment, we will normalize each document's TF-IDF vector to be a unit vector. 

In [5]:
tf_idf = normalize(tf_idf)

We can check that the length (Euclidean norm) of each row is now 1.0, as expected.

In [6]:
for i in range(5):
    doc = tf_idf[i]
    print(np.linalg.norm(doc.todense()))

1.0
1.0
1.0
1.0
1.0


## EM in high dimensions

EM for high-dimensional data requires some special treatment:
 * E step and M step must be vectorized as much as possible, as explicit loops are dreadfully slow in Python.
 * All operations must be cast in terms of sparse matrix operations, to take advantage of computational savings enabled by sparsity of data.
 * Initially, some words may be entirely absent from a cluster, causing the M step to produce zero mean and variance for those words.  This means any data point with one of those words will have 0 probability of being assigned to that cluster since the cluster allows for no variability (0 variance) around that count being 0 (0 mean). Since there is a small chance for those words to later appear in the cluster, we instead assign a small positive variance (~1e-10). Doing so also prevents numerical overflow.
 
We provide the complete implementation for you in the file `em_utilities.py`. For those who are interested, you can read through the code to see how the sparse matrix implementation differs from the previous assignment. 

You are expected to answer some quiz questions using the results of clustering.

**Initializing mean parameters using k-means**

Recall from the lectures that EM for Gaussian mixtures is very sensitive to the choice of initial means. With a bad initial set of means, EM may produce clusters that span a large area and are mostly overlapping. To eliminate such bad outcomes, we first produce a suitable set of initial means by using the cluster centers from running k-means.  That is, we first run k-means and then take the final set of means from the converged solution as the initial means in our EM algorithm.

In [7]:
from sklearn.cluster import KMeans

np.random.seed(5)
num_clusters = 25

# Use scikit-learn's k-means to simplify workflow
#kmeans_model = KMeans(n_clusters=num_clusters, n_init=5, max_iter=400, random_state=1, n_jobs=-1) # uncomment to use parallelism -- may break on your installation
kmeans_model = KMeans(n_clusters=num_clusters, n_init=5, max_iter=400, random_state=1, n_jobs=1)
kmeans_model.fit(tf_idf)
centroids, cluster_assignment = kmeans_model.cluster_centers_, kmeans_model.labels_

means = [centroid for centroid in centroids]

**Initializing cluster weights**

We will initialize each cluster weight to be the proportion of documents assigned to that cluster by k-means above.

In [14]:
num_docs = tf_idf.shape[0]
weights = []
for i in xrange(num_clusters):
    # Compute the number of data points assigned to cluster i:
    num_assigned = len(centroids[i]) # YOUR CODE HERE
    w = float(num_assigned) / num_docs
    weights.append(w)

**Initializing covariances**

To initialize our covariance parameters, we compute $\hat{\sigma}_{k, j}^2 = \sum_{i=1}^{N}(x_{i,j} - \hat{\mu}_{k, j})^2$ for each feature $j$.  For features with really tiny variances, we assign 1e-8 instead to prevent numerical instability. We do this computation in a vectorized fashion in the following code block.

In [15]:
covs = []
for i in xrange(num_clusters):
    member_rows = tf_idf[cluster_assignment==i]
    cov = (member_rows.multiply(member_rows) - 2*member_rows.dot(diag(means[i]))).sum(axis=0).A1 / member_rows.shape[0] \
          + means[i]**2
    cov[cov < 1e-8] = 1e-8
    covs.append(cov)

**Running EM**

Now that we have initialized all of our parameters, run EM.

In [46]:
out = EM_for_high_dimension(tf_idf, means, covs, weights, cov_smoothing=1e-10)

In [47]:
out['loglik']

[3879327582.0772448, 4883345753.5331306, 4883345753.5331306]

In [60]:
print out

{'loglik': [3879327582.0772448, 4883345753.5331306, 4883345753.5331306], 'resp': array([[ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]]), 'covs': [array([  1.00000000e-10,   1.00000000e-10,   1.00000000e-10, ...,
         6.51877866e-09,   3.35273163e-09,   1.39388715e-09]), array([  1.00000000e-10,   1.00000000e-10,   1.00000000e-10, ...,
         9.03175914e-09,   2.98528202e-09,   1.02418143e-09]), array([  1.00000000e-10,   1.00000000e-10,   1.00000000e-10, ...,
         8.11103040e-09,   1.40877532e-09,   9.29567270e-10]), array([  1.00000000e-10,   1.00000000e-10,   1.00000000e-10, ...,
         6.40751196e-09,   2.59161887e-09,   8.83380254e-10]), array([  1.00000000e-10,   1.00000000e-10,   1.00000000e-10, ...,
         8.04873735e-09,   2.39484591e-09,   1.19

## Interpret clustering results

In contrast to k-means, EM is able to explicitly model clusters of varying sizes and proportions. The relative magnitude of variances in the word dimensions tell us much about the nature of the clusters.

Write yourself a cluster visualizer as follows.  Examining each cluster's mean vector, list the 5 words with the largest mean values (5 most common words in the cluster). For each word, also include the associated variance parameter (diagonal element of the covariance matrix). 

A sample output may be:
```
==========================================================
Cluster 0: Largest mean parameters in cluster 

Word        Mean        Variance    
football    1.08e-01    8.64e-03
season      5.80e-02    2.93e-03
club        4.48e-02    1.99e-03
league      3.94e-02    1.08e-03
played      3.83e-02    8.45e-04
...
```

In [48]:
map_index_to_word.print_rows(num_rows=200)

+---------+----------------------------+-------+
| feature |          category          | index |
+---------+----------------------------+-------+
| feature |         conflating         |   0   |
| feature |          diamono           |   1   |
| feature |          bailouts          |   2   |
| feature |        electionruss        |   3   |
| feature |          maywoods          |   4   |
| feature |          feduring          |   5   |
| feature |         spiderbait         |   6   |
| feature |            mcin            |   7   |
| feature |         sumiswald          |   8   |
| feature |           quinta           |   9   |
| feature |            749             |   10  |
| feature |        firedheading        |   11  |
| feature |          hatchett          |   12  |
| feature |          drumsshe          |   13  |
| feature |         2001prior          |   14  |
| feature |        selingerborn        |   15  |
| feature |            lavo            |   16  |
| feature |         

In [66]:
_means = out["means"]
num_clusters = len(_means)
for c in xrange(num_clusters):
    print np.argsort(_means[c][::-1])

[50140 66602 66601 ...,  1464  2327  1237]
[50140 65444 65443 ...,   108    83    90]
[50140 66274 66273 ...,   150   191   300]
[50140 66398 66397 ...,   832  1103   425]
[50140 66196 66195 ...,   145   153   796]
[50140 66674 66673 ...,  1050   134  1586]
[50140 66254 66253 ...,   371  3850   483]
[50140 66357 66356 ...,   351   409   162]
[100281  63810  63809 ...,   7075   4556    892]
[50140 65704 65703 ...,   519   254   775]
[50140 66322 66321 ...,   289   789   291]
[50140 65236 65234 ...,   158    24   197]
[50140 66455 66454 ...,   831   430   250]
[50140 66595 66594 ...,  1445  1720   379]
[50140 66463 66462 ...,   161   150   673]
[50140 66632 66631 ...,  3490   300  1146]
[50140 66655 66654 ...,  4010  2340   815]
[50140 49773 49772 ...,    69   245    21]
[50140 66069 66068 ...,  1380   837   108]
[50140 66620 66619 ...,  1739   598    90]
[50140 66400 66399 ...,   231   938   451]
[50140 66263 66262 ...,   199   248   433]
[50140 66180 66179 ...,   559   162   538]
[5014

In [71]:
# Fill in the blanks
def visualize_EM_clusters(tf_idf, means, covs, map_index_to_word):
    print('')
    print('==========================================================')

    num_clusters = len(means)
    for c in xrange(num_clusters):
        print('Cluster {0:d}: Largest mean parameters in cluster '.format(c))
        print('\n{0: <12}{1: <12}{2: <12}'.format('Word', 'Mean', 'Variance'))
        
        # The k'th element of sorted_word_ids should be the index of the word 
        # that has the k'th-largest value in the cluster mean. Hint: Use np.argsort().
        sorted_word_ids = np.argsort(means[c][::])

        for i in sorted_word_ids[-5:]:
            print '{0: <12}{1:<10.2e}{2:10.2e}'.format(map_index_to_word['category'][i], 
                                                       means[c][i],
                                                       covs[c][i])
        print '\n=========================================================='

In [72]:
'''By EM'''
visualize_EM_clusters(tf_idf, out['means'], out['covs'], map_index_to_word)


Cluster 0: Largest mean parameters in cluster 

Word        Mean        Variance    
literary    4.68e-02    3.29e-03
de          4.77e-02    8.72e-03
poet        5.91e-02    6.36e-03
poems       6.33e-02    6.45e-03
poetry      1.51e-01    1.90e-02

Cluster 1: Largest mean parameters in cluster 

Word        Mean        Variance    
show        1.27e-02    7.33e-04
actress     1.52e-02    1.14e-03
music       1.53e-02    1.04e-03
her         1.04e-01    3.20e-03
she         1.60e-01    4.59e-03

Cluster 2: Largest mean parameters in cluster 

Word        Mean        Variance    
played      3.79e-02    9.46e-04
season      5.06e-02    2.35e-03
league      5.72e-02    2.83e-03
club        5.84e-02    2.55e-03
football    7.45e-02    4.57e-03

Cluster 3: Largest mean parameters in cluster 

Word        Mean        Variance    
house       3.65e-02    2.07e-03
democratic  3.72e-02    2.46e-03
senate      5.04e-02    5.21e-03
republican  5.47e-02    4.55e-03
district    5.56e-02    4.00e

**Quiz Question**. Select all the topics that have a cluster in the model created above. [multiple choice]

## Comparing to random initialization

Create variables for randomly initializing the EM algorithm. Complete the following code block.

In [89]:
print np.random.randn(5)*5

[  8.66001867   0.75293569  -2.78697115  13.70167582  -6.13612759]


In [97]:
np.random.seed(5) # See the note below to see why we set seed=5.
num_clusters = len(means)
num_docs, num_words = tf_idf.shape

random_means = []
random_covs = []
random_weights = []

for k in range(num_clusters):
    
    # Create a numpy array of length num_words with random normally distributed values.
    # Use the standard univariate normal distribution (mean 0, variance 1).
    # YOUR CODE HERE
    mean = np.random.randn(num_words)
    
    # Create a numpy array of length num_words with random values uniformly distributed between 1 and 5.
    # YOUR CODE HERE
    cov = 5*np.random.randn(num_words) + 1.

    # Initially give each cluster equal weight.
    # YOUR CODE HERE
    weight = 0.1
    
    random_means.append(mean)
    random_covs.append(cov)
    random_weights.append(weight)

In [98]:
print random_means
print random_covs
print random_weights

[array([ 0.44122749, -0.33087015,  2.43077119, ..., -0.34219415,
       -1.57035633, -0.49190002]), array([-1.15593615, -0.307251  ,  0.35846375, ...,  0.17532613,
       -0.19471828, -0.86139313]), array([ -1.17264022e-03,  -3.13909586e-01,   2.99073516e-02, ...,
        -1.53438461e+00,  -2.43063811e+00,  -8.87695423e-01]), array([ 0.09222056,  1.78278706, -0.47566219, ..., -0.99481136,
       -1.02602908, -0.24732317]), array([-0.78082584, -1.02219346, -0.56880082, ...,  1.37247112,
        1.2696464 , -1.47166962]), array([ 0.75436271,  0.04673922,  0.7738681 , ...,  2.57508589,
       -0.36601889, -1.60090859]), array([-2.53097684,  0.77194809,  1.3894362 , ...,  1.08989018,
        0.0651108 , -0.26076514]), array([-0.91256716,  0.77017385, -0.56454252, ..., -0.06655384,
       -0.28565974,  0.23027014]), array([-0.91126313, -1.19029827, -1.52965064, ..., -1.80314841,
        0.5487375 ,  0.72153807]), array([ 1.32298234, -1.5554891 , -1.37455741, ..., -1.3175054 ,
        1.8254

**Quiz Question**: Try fitting EM with the random initial parameters you created above. (Use `cov_smoothing=1e-5`.) Store the result to `out_random_init`. What is the final loglikelihood that the algorithm converges to? 

In [99]:
out_random_init = EM_for_high_dimension(tf_idf, random_means, random_covs, random_weights, cov_smoothing=1e-5)
out_random_init['loglik']

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

**Quiz Question:** Is the final loglikelihood larger or smaller than the final loglikelihood we obtained above when initializing EM with the results from running k-means?

**Quiz Question**: For the above model, `out_random_init`, use the `visualize_EM_clusters` method you created above. Are the clusters more or less interpretable than the ones found after initializing using k-means?

In [ ]:
# YOUR CODE HERE. Use visualize_EM_clusters, which will require you to pass in tf_idf and map_index_to_word.
...

**Note**: Random initialization may sometimes produce a superior fit than k-means initialization. We do not claim that random initialization is always worse. However, this section does illustrate that random initialization often produces much worse clustering than k-means counterpart. This is the reason why we provide the particular random seed (`np.random.seed(5)`).

## Takeaway

In this assignment we were able to apply the EM algorithm to a mixture of Gaussians model of text data. This was made possible by modifying the model to assume a diagonal covariance for each cluster, and by modifying the implementation to use a sparse matrix representation. In the second part you explored the role of k-means initialization on the convergence of the model as well as the interpretability of the clusters.